In [16]:
import turtle
import numpy as np

In [3]:
class World:
    
    def __init__(self, mx, my):
        
        self._maxX = mx
        self._maxY = my
        self._thingList = []
        self._grid = np.array([None] * (self._maxX * self._maxY)).reshape(self._maxX, self._maxY)
        self.bearCount = 0
        self.fishCount = 0
            
        Bear = "\U0001F43B"
        Fish = "\U0001F41F"
        
        self.wturtle = turtle.Turtle()
        self.wscreen = turtle.Screen()
        self.wscreen.setworldcoordinates(0, 0, self._maxX - 1, self._maxY - 1)
        self.wturtle.hideturtle()
        
    def draw(self):
        
        self.wscreen.tracer(0)
        self.wturtle.forward(self._maxX - 1)
        self.wturtle.left(90)
        self.wturtle.forward(self._maxY - 1)
        self.wturtle.left(90)
        self.wturtle.forward(self._maxX - 1)
        self.wturtle.left(90)
        self.wturtle.forward(self._maxY - 1)
        self.wturtle.left(90)
        
        for i in range(self._maxY - 1):
            self.wturtle.forward(self._maxX - 1)
            self.wturtle.backward(self._maxX - 1)
            self.wturtle.left(90)
            self.wturtle.forward(1)
            self.wturtle.right(90)
        
        self.wturtle.forward(1)
        self.wturtle.right(90)
        
        for i in range(self._maxX - 2):
            self.wturtle.forward(self._maxY - 1)
            self.wturtle.backward(self._maxY - 1)
            self.wturtle.left(90)
            self.wturtle.forward(1)
            self.wturtle.right(90)
            
        self.wscreen.tracer(1)
        
    def __getitem__(self, loc):
        
        return self._grid(loc)
    
    def __setitem__(self, loc, x):
        
        pass
        
    def freezeWorld(self):
        
        self.wscreen.exitonclick()
        
    def addThing(self, thing, x, y):
        
        thing.setX(x)
        thing.setY(y)
        self._grid[y, x] = thing
        thing.setWorld(self)
        self.thingList.append(thing)
        thing.appear()
        if isinstance(thing, Bear):
            thing.incBears()
        else:
            thing.incFish()
        
    def delThing(self, thing):
        
        thing.hide()
        self._grid[thing.getY()][thing.getX()] = None
        self.thingList.remove(thing)
        if isinstance(thing, Bear):
            thing.decBears()
        else:
            thing.decFish()
        
    def moveThing(self, oldx, oldy, newx, newy):
        
        self._grid[newy][newx] = self._grid[oldy][oldx]
        self._grid[oldy][oldx] = None
        
    def getMaxX(self):
        
        return self._maxX
    
    def getMaxY(self):
        
        return self._maxY
    
    def liveALittle(self): #chooses a random fish or bear to live during that time period
        
        if self.thingList != []:
            thing = random.randrange(len(self.thingList))
            randomthing = self.thingList[thing]
            randomthing.liveALittle()
            
    def emptyLocation(self, x, y):
        
        if self._grid[y][x] == None:
            return True
        else:
            return False
        
    def lookAtLocation(self, x, y):
        
        return self._grid[y][x]
    
    def getNumBears(self):
        
        return self.bearCount
    
    def getNumFish(self):
        
        return self.fishCount
    
    def incBears(self):
        
        self.bearCount += 1
                    
    def decBears(self):
        
        self.bearCount -= 1
        
    def incFish(self):
        
        self.fishCount += 1
        
    def decFish(self):
        
        self.fishCount -= 1
        
    def showCounts(self, animal):
        
        pass

In [7]:
class Fish:
    
    def __init__(self):
        
        self.turtle = turtle.Turtle()
        self.turtle.up()
        self.turtle.hideturtle()
                
        self.xpos = 0
        self.ypos = 0
        self.world = None        
        self.breedTick = 0
        offsetList = [(-1,1), (0,1), (1,1)
                      (-1,0)         (1,0)
                      (-1,-1), (0,-1), (1,-1)]
        
    def setX(self, newX):
        
        self.xpos = newX
        
    def setY(self, newY):
        
        self.ypos = newY
        
    def getX(self):
        
        return self.xpos
    
    def getY(self):
        
        return self.ypos
    
    def setWorld(self, aworld):
        
        self.world = aworld
        
    def appear(self):
        
        self.turtle.goto(self.xpos, self.ypos)
        self.turtle.showturtle()
        
    def hide(self):
        
        self.turtle.hideturtle()
        
    def move(self, newX, newY):
        
        self.world.moveThing(self.xpos, self.ypos, newX, newY)
        self.xpos = newX
        self.ypos = newY
        self.turtle.goto(self.xpos, self.ypos)
        
    def liveALittle(self):
        
        adjFish = 0
        for offset in offsetList:
            newx = self.xpos + offset[0]
            newy = self.ypos + offset[1]
            if 0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY():
                    if (not self.world.emptyLocation(newx, newy)) and isinstance(self.world.lookAtLocation(newx, newy), Fish):
                            adjFish += 1
                            
        if adjFish >= 2:
            self.world.delThing(self)
        else:
            self.breedTick += 1
            if self.breedTick >= 12:
                self.tryToBreed()
                
            self.tryToMove()
            
    def tryToBreed(self):
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            childThing = Fish()
            self.world.addThing(childThing, nextx, nexty)
            self.breedTick = 0
            
    def tryToMove(self):
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            self.move(nextx, nexty)

In [8]:
class Bear:
    
    def __init__(self):
        
        self.turtle = turtle.Turtle()
        self.turtle.up()
        self.turtle.hideturtle()
        self.turtle.shape("\U0001F43B")
        
        self.xpos = 0
        self.ypos = 0
        self.world = None
        
        self.breedTick = 0
        self.starveTick = 0
        
    def setX(self, newX):
        
        self.xpos = newX
        
    def setY(self, newY):
        
        self.ypos = newY
        
    def getX(self):
        
        return self.xpos
    
    def getY(self):
        
        return self.ypos
    
    def setWorld(self, world):
        
        self.world = world
        
    def appear(self):
        
        self.turtle.goto(self.xpos, self.ypos)
        self.turtle.showturtle()
        
    def hide(self):
        
        self.turtle.hideturtle()
        
    def move(self, newX, newY):
        
        self.world.moveThing(self.xpos, self.ypos, newX, newY)
        self.xpos = newX
        self.ypos = newY
        self.turtle.goto(self.xpos, self.ypos)
        
    def liveALittle(self):
        
        self.breedTick += 1
        if self.breedTick >= 8:
            self.tryToBreed()
            
        self.tryToEat()
        
        if self.starveTick == 10:
            self.world.delThing(self)
        else:
            self.tryToMove()
            
    def tryToBreed(self):
        
        offsetList = [(-1,1), (0,1), (1,1)
                      (-1,0)         (1,0)
                      (-1,-1), (0,-1), (1,-1)]
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            childThing = Bear()
            self.world.addThing(childThing, nextx, nexty)
            self.breedTick = 0
            
    def tryToMove(self):
        
        offsetList = [(-1,1), (0,1), (1,1)
                      (-1,0)         (1,0)
                      (-1,-1), (0,-1), (1,-1)]
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            self.move(nextx, nexty)
            
    def tryToEat(self):
        
        offsetList = [(-1,1), (0,1), (1,1)
                      (-1,0)         (1,0)
                      (-1,-1), (0,-1), (1,-1)]
        
        adjprey = []
        
        for offset in offsettList:
                newx = self.xpos + offset[0]
                newy = self.ypos + offset[1]
                if 0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY():
                    if (not self.world.emptyLocation(newx, newy)) and isinstance(self.world.lookAtLocation(newx, newy), Fish):
                        adjprey.append(self.world.lookAtLocation(newx, newy))
                        
        if len(adjprey) > 0:
            randomprey = adjprey[random.randrange(len(adjprey))]
            preyx = randomprey.getX()
            preyy = randomprey.getY()
            
            self.world.delThing(randomprey)
            self.move(preyx, preyy)
            self.starveTick = 0
        else:
            self.starveTick += 1

In [11]:
Fish()

TypeError: 'tuple' object is not callable

In [11]:
def mainSimulation():
    
    numberOfBears = 10
    numberOfFish = 10
    worldLifeTime = 2500
    worldWidth = 50
    worldHeight = 25
    
    myworld = World(worldWidth, worldHeight)
    myworld.draw()
    
    for i in range(numberOfFish):
        newfish = Fish()
        x = random.randrange(myworld.getMaxX())
        y = random.randrange(myworld.getMaxY())
        while not myworld.emptyLocation(x, y):
            x = random.randrange(myworld.getMaxX())
            y = random.randrange(myworld.getMaxY())
        myworld.addThing(newfish, x, y)
    
    for i in range(numberOfBears):
        newbear = Bear()
        x = random.randrange(myworld.getMaxX())
        y = random.randrange(myworld.getMaxY())
        while not myworld.emptyLocation(x, y):
            x = random.randrange(myworld.getMaxX())
            y = random.randrange(myworld.getMaxY())
        myworld.addThing(newbear, x, y)
        
    for i in range(worldLifeTime):
        myworld.liveALittle()
        
    myworld.freezeWorld()

In [ ]:
mainSimulation()

In [7]:
print("\U0001F41F")
print("\U0001F43B")

🐟
🐻


In [12]:
class Animal:
    
    def __init__(self):
        
        self.turtle = turtle.Turtle()
        self.turtle.up()
        self.turtle.hideturtle()        
        self.xpos = 0
        self.ypos = 0
        self.world = None        
        self.breedTick = 0
        offsetList = [(-1,1), (0,1), (1,1)
                      (-1,0)         (1,0)
                      (-1,-1), (0,-1), (1,-1)]
        
    def setX(self, newX):
        
        self.xpos = newX
        
    def setY(self, newY):
        
        self.ypos = newY
        
    def getX(self):
        
        return self.xpos
    
    def getY(self):
        
        return self.ypos
    
    def setWorld(self, aworld):
        
        self.world = aworld
        
    def appear(self):
        
        self.turtle.goto(self.xpos, self.ypos)
        self.turtle.showturtle()
        
    def hide(self):
        
        self.turtle.hideturtle()
        
    def move(self, newX, newY):
        
        self.world.moveThing(self.xpos, self.ypos, newX, newY)
        self.xpos = newX
        self.ypos = newY
        self.turtle.goto(self.xpos, self.ypos)
        
    def tryToBreed(self):
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            childThing = Bear()
            self.world.addThing(childThing, nextx, nexty)
            self.breedTick = 0
            
    def tryToMove(self):
        
        randomOffsetIndex = random.randrange(len(offsetList))
        randomOffset = offsetList[randomOffsetIndex]
        
        nextx = self.xpos + randomOffset[0]
        nexty = self.ypos + randomOffset[1]
        
        while not (0 <= nextx < self.world.getMaxX() and 0 <= nexty < self.world.getMaxY()):
            ranodmOffsetIndex = random.randrange(len(offsetList))
            randomOffset = offsetList[randomOffsetIndex]
            nextx = self.xpos + randomOffset[0]
            nexty = self.ypos + randomOffset[1]
            
        if self.world.emptyLocation(nextx, nexty):
            self.move(nextx, nexty)

In [23]:
class Bear(Animal):
    
    def __init__(self, starveTime=10, breedTime=8):
        
        super().__init__()
        self.starveTick = 0
        self.starve = starveTime
        self.breed = breedTime
        
    def tryToEat(self):
        
        adjprey = []
        
        for offset in offsettList:
                newx = self.xpos + offset[0]
                newy = self.ypos + offset[1]
                if 0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY():
                    if (not self.world.emptyLocation(newx, newy)) and isinstance(self.world.lookAtLocation(newx, newy), Fish):
                        adjprey.append(self.world.lookAtLocation(newx, newy))
                        
        if len(adjprey) > 0:
            randomprey = adjprey[random.randrange(len(adjprey))]
            preyx = randomprey.getX()
            preyy = randomprey.getY()
            
            self.world.delThing(randomprey)
            self.move(preyx, preyy)
            self.starveTick = 0
        else:
            self.starveTick += 1
            
    def liveALittle(self):
        
        self.breedTick += 1
        if self.breedTick >= self.breed:
            self.tryToBreed()
            
        self.tryToEat()
        
        if self.starveTick == self.starve:
            self.world.delThing(self)
        else:
            self.tryToMove()
            
    def getStarve(self):
        
        return self.starve
    
    def getBreed(self):
        
        return self.breed
    
    def setStarve(self, x):
        
        self.starve = x
        
    def setBreed(self, x):
        
        self.breed = x

In [24]:
class Fish(Animal):
    
    def __init__(self, crowd=2, breedInterval=12):
        
        super().__init__()
        self.overcrowd = crowd
        self.breedInterval = breedInterval
        
    def liveALittle(self):
        
        adjFish = 0
        for offset in offsetList:
            newx = self.xpos + offset[0]
            newy = self.ypos + offset[1]
            if 0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getMaxY():
                    if (not self.world.emptyLocation(newx, newy)) and isinstance(self.world.lookAtLocation(newx, newy), Fish):
                            adjFish += 1
                            
        if adjFish >= self.overcrowd:
            self.world.delThing(self)
        else:
            self.breedTick += 1
            if self.breedTick >= self.breedInterval:
                self.tryToBreed()
                
            self.tryToMove()
            
    def getOvercrowd(self):
        
        return self.overcrowd
    
    def getBreedInterval(self):
        
        return self.breedInterval
    
    def setOvercrowd(self, x):
        
        self.overcrowd = x
        
    def setBreedInterval(self, x):
        
        self.breedInterval = x

In [29]:
grid = np.array([None] * 100).reshape(10,10)

In [30]:
grid

array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [35]:
a = np.array(range(20)).reshape(4,5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [37]:
a[0][2]

2

In [39]:
loc = (1,4)
a[loc]

9

In [40]:
a[3,0]

15

In [41]:
type(a)

numpy.ndarray

In [57]:
def grid(x,y):
    return np.array([None] * (x*y)).reshape(x,y)

In [58]:
x = grid(10,10)

In [59]:
x[2, 5] = '\U0001F43B'

In [60]:
x

array([[None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, '🐻', None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None]], dtype=object)

In [3]:
grid = np.array([None] * 25).reshape(5,5)
grid

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

In [11]:
grid[2,2] = '\U0001F43B' + str(12)
grid

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, '🐻12', None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

In [12]:
type(grid[2,2])

str

In [14]:
grid[2,2][0] == '\U0001F43B'

True

In [20]:
v = np.array([None] * 25)
grid = v.reshape(5,5)
print(v)
print(grid)

[None None None None None None None None None None None None None None None
 None None None None None None None None None None]
[[None None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]]


In [22]:
v[0] = "\U0001F41F"
print(v)
print(grid)

['🐟' None None None None None None None None None None None None None None
 None None None None None None None None None None]
[['🐟' None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]]


In [23]:
len(v)

25

In [19]:
class Jacob:
    
    def __init__(self):
        
        self._name = "Jacob"
        self._age = "23"
        self._major = "CIS"
        self._grid = np.array([None] * 25).reshape(5,5)
        
    def __repr__(self):
        
        return "%s" % (self._grid)

In [20]:
Jacob()

[[None None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]
 [None None None None None]]